# HANDS-ON 02

2023-04-09

## The Laplace transform

The Laplace transform is an integral transform that converts a function of a real variable, usually $t$ in the time domain, into a function of a complex variable $s$ in the complex frequency domain, and has many applications in science and engineering as a tool for solving differential equations, including transforming ordinary differential equations into algebraic equations and convolution into multiplication ([Wikipedia](https://en.wikipedia.org/wiki/Laplace_transform)).

Considering a function $f$, one can define the Laplace transform as the integral

$ \displaystyle \mathcal L \{f\}(s) = \int_0^\infty f(t) e^{-st} dt $

## Sympy

[SymPy](https://en.wikipedia.org/wiki/SymPy) is an open source Python library for symbolic computing, providing computer algebra capabilities, written entirely in Python with few dependencies, easy to access, and with a simple and extensible code base. The Sympy library was used to solve most of the problems in this work, and some *methods* are:

**laplace_transform** : compute the Laplace Transform $F(s)$ of $f(t)$, defined as

$ \displaystyle F(s) = \int_{0^{-}}^\infty e^{-st} f(t) \mathrm{d}t $

**inverse_laplace_transform** : compute the inverse Laplace transform of $ F(s) $, defined as

$  \displaystyle f(t) = \frac{1}{2\pi i} \int_{c-i\infty}^{c+i\infty} e^{st} F(s) \mathrm{d}s $

**dsolve** : solves any (supported) kind of ordinary differential equation and system of ordinary differential equations. You can specify the set of initial conditions (IC) or boundary conditions for the differential equation.

**apart** : performs a partial fraction decomposition on a rational function.

**solve** : solves algebraic equations, assuming all equations equal 0.

**subs** : replaces all instances of something in an expression with something else.

## Load libraries

In [3]:
import sympy as sp, math as mt

x, s, t = sp.symbols('x, s, t')
k = sp.symbols('k', real=True, positive=True)
f = sp.symbols('f', cls=sp.Function)

sp.__version__

'1.11.1'

## Inverse Laplace transform

For each of the problems, find the inverse Laplace transform of the given function

### Exercise "2.1"

$ \displaystyle \frac {2} {s^2 + 2s + 5} $

Equivalent Python/Sympy expression:

In [7]:
f = 2 / (s**2 + 2 * s + 5)
f

2/(s**2 + 2*s + 5)

Compute the inverse Laplace transform:

In [8]:
sp.inverse_laplace_transform(f, s, t)

exp(-t)*sin(2*t)*Heaviside(t)

where $\theta$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function) whose value is zero for negative arguments and one for positive. It is used to force the result to be equal to zero before $t = 0$, following the the Laplace transform rules.

### Exercise 2.2

$ \displaystyle \frac {2s - 3} {s^2 -4} $

In [24]:
f = (2 * s - 3) / (s**2 - 4)
f

(2*s - 3)/(s**2 - 4)

In [25]:
sp.inverse_laplace_transform(f, s, t)

(exp(4*t) + 7)*exp(-2*t)*Heaviside(t)/4

### Exercise 2.3

$ \displaystyle \frac {1 - 2s} {s^2 + 4s + 5} $

In [11]:
f = (1 - 2 * s) / (s**2 + 4 * s + 5)
f

(1 - 2*s)/(s**2 + 4*s + 5)

In [12]:
sp.inverse_laplace_transform(f, s, t)

(5*sin(t) - 2*cos(t))*exp(-2*t)*Heaviside(t)

## Solving manually, and using Sympy

### Exercise 2.4

$ \displaystyle \frac {d^2 f(x)} {dx^2} + f(x) = 0 \ , \ \ f(0)=1, f'(0) = 0 $

#### Solving manually

$f''(x) + f(x) = 0$

$ L[f''(x)] + L[f(x)] = 0 $

$s^2 L [f(x)] - s f(0) - f'(0) + L [f(x)] = 0 $

substituting the initial conditions into the equation:

$s^2 L [f(x)] - s + L [f(x)] = 0 $

rearranging:

$ L[f(x)](s^2 + 1) = s $

$ L[f(x)] = \displaystyle \frac {s} {s^2 + 1} $

using the Table Of Laplace Transforms:

$ f(x) = \cos (x) $

#### Solving using Sympy

In [62]:
ex = f(x).diff(x, x) + f(x)
ex

f(x) + Derivative(f(x), (x, 2))

In [63]:
ex2 = sp.laplace_transform(ex, x, s)
ex2

s**2*LaplaceTransform(f(x), x, s) - s*f(0) + LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), x), x, 0)

Substitute the boundary conditions into the equation:

In [64]:
ex3 = ex2.subs({f(0): 1})
ex3

s**2*LaplaceTransform(f(x), x, s) - s + LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), x), x, 0)

In [65]:
ex4 = ex3.subs({sp.diff(f(x), x).subs(x,0): 0})
ex4

s**2*LaplaceTransform(f(x), x, s) - s + LaplaceTransform(f(x), x, s)

Expression of $f(x)$:

In [10]:
ef = sp.laplace_transform(f(x), x, s)
ef

LaplaceTransform(f(x), x, s)

In [71]:
ex5 = ex4.apart(ef)
ex5

-s + (s**2 + 1)*LaplaceTransform(f(x), x, s)

Rearranges the equation, putting $f(x)$ on the left hand side (lhs) and the rest of the equation on the right hand side (rhs) of the expression. In the cell below, let's work with rhs:

In [73]:
ex6 = sp.solve(ex5, ef)
ex6[0]

s/(s**2 + 1)

Calculate the inverse Laplace transform, and add $f(x)$ to the lhs:

In [69]:
sp.Eq(f(x), sp.inverse_laplace_transform(ex6[0], s, x))

Eq(f(x), cos(x)*Heaviside(x))

where $\theta$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function)

#### Solving using *dsolve*, to check the results:

In [16]:
sp.dsolve(ex, f(x), ics={f(0): 1, sp.diff(f(x), x).subs(x,0): 0})

Eq(f(x), cos(x))

## Laplace transform using Sympy

In each of the problems, use the Laplace transform to solve the given initial value (IC) problem

### Exercise 2.5

$ y'' - 4y' + 4y = 0 \ , \ \ \ $ IC: $\ y(0)=1, y'(0)=1 $

In [4]:
y = f(x).diff(x, x) - 4 * f(x).diff(x) + 4 * f(x)
y

4*f(x) - 4*Derivative(f(x), x) + Derivative(f(x), (x, 2))

In [5]:
F = sp.laplace_transform(y, x, s)
F

s**2*LaplaceTransform(f(x), x, s) - 4*s*LaplaceTransform(f(x), x, s) - s*f(0) + 4*LaplaceTransform(f(x), x, s) + 4*f(0) - Subs(Derivative(f(x), x), x, 0)

In [6]:
F2 = F.subs({f(0): 1})
F2

s**2*LaplaceTransform(f(x), x, s) - 4*s*LaplaceTransform(f(x), x, s) - s + 4*LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), x), x, 0) + 4

In [7]:
F3 = F2.subs({sp.diff(f(x), x).subs(x,0): 1})
F3

s**2*LaplaceTransform(f(x), x, s) - 4*s*LaplaceTransform(f(x), x, s) - s + 4*LaplaceTransform(f(x), x, s) + 3

In [9]:
F4 = F3.apart(sp.laplace_transform(f(x), x, s))
F4

-s + (s**2 - 4*s + 4)*LaplaceTransform(f(x), x, s) + 3

Rearranges the equation, separating $f(x)$ :

In [17]:
F5 = sp.solve(F4, ef)[0]
F5

(s - 3)/(s**2 - 4*s + 4)

Calculate the inverse Laplace transform, and add $f(x)$ to the lhs:

In [18]:
sp.Eq(f(x), sp.inverse_laplace_transform(F5, s, x))

Eq(f(x), (1 - x)*exp(2*x)*Heaviside(x))

#### Solving using *dsolve*, to check the results:

In [4]:
e25 = sp.dsolve(F, f(x), ics={f(0): 1, sp.diff(f(x), x).subs(x, 0): 1})
e25

Eq(f(x), (1 - x)*exp(2*x))

### Exercise 2.6

$ y^{iv} - 4y = 0 \ , \ \ y(0)=1, \ y'(0)=0, \ y''(0)=-2, \ y'''(0)=0 $

In [20]:
y = f(x).diff(x, x, x, x) - 4 * f(x)
y

-4*f(x) + Derivative(f(x), (x, 4))

In [21]:
F = sp.laplace_transform(y, x, s)
F

s**4*LaplaceTransform(f(x), x, s) - s**3*f(0) - s**2*Subs(Derivative(f(x), x), x, 0) - s*Subs(Derivative(f(x), (x, 2)), x, 0) - 4*LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), (x, 3)), x, 0)

Substitute the boundary conditions into the equation:

In [22]:
F2 = F.subs({f(0): 1})
F2

s**4*LaplaceTransform(f(x), x, s) - s**3 - s**2*Subs(Derivative(f(x), x), x, 0) - s*Subs(Derivative(f(x), (x, 2)), x, 0) - 4*LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), (x, 3)), x, 0)

In [23]:
F3 = F2.subs({sp.diff(f(x), x).subs(x, 0): 0})
F3

s**4*LaplaceTransform(f(x), x, s) - s**3 - s*Subs(Derivative(f(x), (x, 2)), x, 0) - 4*LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), (x, 3)), x, 0)

In [24]:
F4 = F3.subs({sp.diff(f(x), x, x).subs(x, 0): -2})
F4

s**4*LaplaceTransform(f(x), x, s) - s**3 + 2*s - 4*LaplaceTransform(f(x), x, s) - Subs(Derivative(f(x), (x, 3)), x, 0)

In [25]:
F5 = F4.subs({sp.diff(f(x), x, x, x).subs(x, 0): 0})
F5

s**4*LaplaceTransform(f(x), x, s) - s**3 + 2*s - 4*LaplaceTransform(f(x), x, s)

Performs a partial fraction decomposition:

In [26]:
F6 = F5.apart(sp.laplace_transform(f(x), x, s))
F6

-s**3 + 2*s + (s**4 - 4)*LaplaceTransform(f(x), x, s)

Rearranges the equation:

In [27]:
F7 = sp.solve(F6, ef)[0]
F7

s/(s**2 + 2)

Compute the inverse Laplace transform:

In [28]:
sp.Eq(f(x), sp.inverse_laplace_transform(F7, s, x))

Eq(f(x), cos(sqrt(2)*x)*Heaviside(x))

where $\theta$ is the [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function)

#### Solving using *dsolve*, to check the results:

In [6]:
e26 = sp.dsolve(y,
                f(x),
                ics={
                    f(0): 1,
                    sp.diff(f(x), x).subs(x, 0): 0,
                    sp.diff(f(x), x, x).subs(x, 0): -2,
                    sp.diff(f(x), x, x, x).subs(x, 0): 0,
                })
e26

Eq(f(x), cos(sqrt(2)*x))

## Modulus "a mod b"

Obtain approximate values for y(e) for equations 2.5 and 2.6 and determine the mod (Python "%") between them: a mod b , with a > b

where "e" is the Euler's constant

### Exercise 2.7

In [16]:
e25e = e25.subs({x: mt.e}).rhs
e25e

-394.606281270100

In [17]:
e26e = e26.subs({x: mt.e}).rhs
e26e

cos(2.71828182845905*sqrt(2))

In [20]:
(a, b) = (e25e, e26e) if e25e > e26e else (e26e, e25e)

In [24]:
sp.Mod(e25.rhs, e26.rhs)

Mod((1 - x)*exp(2*x), cos(sqrt(2)*x))

In [23]:
sp.Mod(a, b).evalf()

-0.763139839653820

# References

- https://docs.sympy.org
- https://dynamics-and-control.readthedocs.io/en/latest/1_Dynamics/3_Linear_systems/Laplace%20transforms.html
- https://www.reddit.com/r/learnpython/comments/diakmu/can_sympy_rearrange_variables/
- https://notebook.community/nicoguaro/AdvancedMath/notebooks/sympy/laplace_transform
- https://www.geeksforgeeks.org/python-sympy-mod-method/
- https://github.com/sympy/sympy/issues/7219#issuecomment-154768904